In [1]:
from pyspark.sql import Row
from pyspark.sql.functions import col 
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import RandomForestClassifier

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
import time

In [2]:
base_path = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_prediction/"

train_positive_pairs = sc.textFile(base_path + "new_positive_pairs_features/day*/part-*").map(lambda x: (eval(x)[0], [abs(i) for y in eval(x)[1] for i in y ]))
train_unlabled_pairs = sc.textFile(base_path + "new_unlabled_pairs_features/part-*").map(lambda x: (eval(x)[0], [abs(i) for y in eval(x)[1] for i in y ]))

test_positive_pairs = sc.textFile(base_path + "test_positive_pairs/part-*")

# test_pairs_features = sc.textFile(base_path + "new_test_pairs_features/part-*").map(lambda x: (eval(x)[0], [abs(i) for y in eval(x)[1] for i in y ]))

test_pairs_features_50w = sc.textFile(base_path + "test_pairs_features_50w/part-*").map(lambda x: eval(x))

In [9]:
test_positive_pairs_list = test_positive_pairs.collect()   # 第四天数据中的正例对
test_data_4thday = test_pairs_features_50w.map(lambda x: Row(label=1.0 if x[0] in test_positive_pairs_list else 0.0,\
                                                         features=Vectors.dense(x[1]))).toDF()

In [6]:
test_pos_features = sc.textFile(base_path + "test_pos_features/part-*").map(lambda x: eval(x))
test_neg_features = sc.textFile(base_path + "test_neg_features_8w/part-*").map(lambda x: eval(x))
test_data_4thday = test_pos_features.map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
        .union(test_neg_features.map(lambda x: Row(label=0.0, features=Vectors.dense(x[1])))).toDF()

In [7]:
y_label = [i.label for i in test_data_4thday.select('label').collect()]

In [8]:
test_data_4thday_length = len(y_label)
print(test_data_4thday_length)

157653


In [4]:
# alfa = 0.15?
train_negative_pairs = sc.textFile(base_path + "wbb_train_negative_pairs-0.15").map(lambda x: eval(x)).collect()

In [5]:

for blta in [0.3]: # 0.6
    with open('qq_PU_blta_1.log', 'a+') as log_file3:
        log_file3.write("\n\n==========blta: " + str(blta) +"============\n")
        log_file3.write(time.asctime() + '\n')
        print(time.asctime() + '\n')
        
        # train_positive_pairs.count() = 235586
        # 从train pos 里面取一部分加到 neg 里面组成PU数据
        pnsplit = int(235586*blta)   # pnsplit以前的部分作为pu_pos, 以后的部分作为pu_neg
        pu_pos_features = train_positive_pairs.collect()[:pnsplit]
        pu_neg_features = train_negative_pairs + train_positive_pairs.collect()[pnsplit:]
        #pu_spy = [Vectors.dense(x[1]) for x in train_positive_pairs.collect()[pnsplit:]]
        
        log_file3.write("length of pu_pos_features: " + str(len(pu_pos_features)) + '\n')
        log_file3.write("length of pu_neg_features: " + str(len(pu_neg_features)) + '\n')
        log_file3.write(time.asctime() + '\n\n')
        print(time.asctime())

        gamma=0.3  # 从PU数据里面取一部分训练，另一部分作为测试
        pu_train_pos = sc.parallelize(pu_pos_features).takeSample(False, int(len(pu_pos_features) * gamma))  # list
        pu_test_pos = sc.parallelize(pu_pos_features).subtractByKey(sc.parallelize(pu_train_pos)).collect()
        log_file3.write("length of pu_train_pos: " + str(len(pu_train_pos)) + '\n')
        log_file3.write("length of pu_test_pos: " + str(len(pu_test_pos)) + '\n')
        pu_train_neg = sc.parallelize(pu_neg_features, 50).takeSample(False, int(len(pu_neg_features) * gamma))  # list
        pu_test_neg = sc.parallelize(pu_neg_features).subtractByKey(sc.parallelize(pu_train_neg)).collect()
        log_file3.write("length of pu_train_neg: " + str(len(pu_train_neg)) + '\n')
        log_file3.write("length of pu_test_neg: " + str(len(pu_test_neg)) + '\n')
        log_file3.write(time.asctime() + '\n\n')
        print(time.asctime())

        pu_train_data = sc.parallelize(pu_train_pos).map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
        .union(sc.parallelize(pu_train_neg).map(lambda x: Row(label=0.0, features=Vectors.dense(x[1])))).toDF()

        pu_test_data = sc.parallelize(pu_test_pos).map(lambda x: Row(label=1.0, features=Vectors.dense(x[1]), pair=x[0]))\
        .union(sc.parallelize(pu_test_neg).map(lambda x: Row(label=0.0, features=Vectors.dense(x[1]), pair=x[0]))).toDF()

        """
        ### LR
        log_file3.write("###### LogisticRegression ######\n")
        publr = LogisticRegression().fit(pu_train_data)
        pu_test_data_preRes_LR = publr.transform(pu_test_data.select("features")).join(pu_test_data, 'features', "left_outer")

        predict_to_0_LR = pu_test_data_preRes_LR.rdd.filter(lambda x: (x.pair.split('-')[0] != 'day123' and x.prediction == 0.0))
        predict_to_0_prob_LR = predict_to_0_LR.map(lambda x: x.probability[0]).collect()
        
        prob_LR_max = max(predict_to_0_prob_LR)
        prob_LR_mean = sum(predict_to_0_prob_LR)/len(predict_to_0_prob_LR)
        print(len(predict_to_0_prob_LR), prob_LR_max, min(predict_to_0_prob_LR), prob_LR_mean)
        log_file3.write("len(predict_to_0_prob_LR) ++ max(predict_to_0_prob_LR) ++ min(predict_to_0_prob_LR) ++ mean(predict_to_0_prob_LR) \n")
        log_file3.write(str(len(predict_to_0_prob_LR)) + ', ' + str(prob_LR_max) + ', ' + str(min(predict_to_0_prob_LR)) + ', ' + str(prob_LR_mean) + '\n')
        log_file3.write(time.asctime() + '\n')
        
        #RN_LR_max = pu_test_data_preRes_LR.rdd.filter(lambda x: x.probability[0] >= prob_LR_max).map(lambda x: x.features).collect()
        RN_LR_mean = pu_test_data_preRes_LR.rdd.filter(lambda x: x.probability[0] > prob_LR_mean).map(lambda x: x.features).collect()
        #log_file3.write('len(RN_LR_max): '+str(len(RN_LR_max))+'\n')
        log_file3.write("len(RN_LR_mean):" + str(len(RN_LR_mean))+'\n')
        #print('len(RN_LR_max): '+str(len(RN_LR_max))+'\n')
        print("len(RN_LR_mean):" + str(len(RN_LR_mean))+'\n')
        #sc.parallelize(RN_LR_max).saveAsTextFile(base_path + "RN_blta/RN_LR_max-" + str(blta))
        sc.parallelize(RN_LR_mean).saveAsTextFile(base_path + "RN_blta/RN_LR_mean-" + str(blta))
              
        ### NB
        log_file3.write("###### NaiveBayes ######\n")
        pubNB = NaiveBayes().fit(pu_train_data)
        pu_test_data_preRes_NB = pubNB.transform(pu_test_data.select("features")).join(pu_test_data, 'features', "left_outer")

        predict_to_0_NB = pu_test_data_preRes_NB.rdd.filter(lambda x: (x.pair.split('-')[0] != 'day123' and x.prediction == 0.0))
        predict_to_0_prob_NB = predict_to_0_NB.map(lambda x: x.probability[0]).collect()
        
        prob_NB_max = max(predict_to_0_prob_NB)
        prob_NB_mean = sum(predict_to_0_prob_NB)/len(predict_to_0_prob_NB)
        print(len(predict_to_0_prob_NB), prob_NB_max, min(predict_to_0_prob_NB), prob_NB_mean)
        log_file3.write("len(predict_to_0_prob_NB) ++ max(predict_to_0_prob_NB) ++ min(predict_to_0_prob_NB) ++ mean(predict_to_0_prob_NB) \n")
        log_file3.write(str(len(predict_to_0_prob_NB)) + ', ' + str(prob_NB_max) + ', ' + str(min(predict_to_0_prob_NB)) + ', ' + str(prob_NB_mean) + '\n')
        log_file3.write(time.asctime() + '\n')
        
        #RN_NB_max = pu_test_data_preRes_NB.rdd.filter(lambda x: x.probability[0] >= prob_NB_max).map(lambda x: x.features).collect()
        RN_NB_mean = pu_test_data_preRes_NB.rdd.filter(lambda x: x.probability[0] > prob_NB_mean).map(lambda x: x.features).collect()
        #log_file3.write('len(RN_NB_max): '+str(len(RN_NB_max))+'\n')
        log_file3.write("len(RN_NB_mean):" + str(len(RN_NB_mean))+'\n')
        #print('len(RN_NB_max): '+str(len(RN_NB_max))+'\n')
        print("len(RN_NB_mean):" + str(len(RN_NB_mean))+'\n')
        #sc.parallelize(RN_NB_max).saveAsTextFile(base_path + "RN_blta/RN_NB_max-" + str(blta))
        sc.parallelize(RN_NB_mean).saveAsTextFile(base_path + "RN_blta/RN_NB_mean-" + str(blta))
        """
        ### DT
        log_file3.write("###### DecisionTreeClassifier ######\n")
        pubDT = DecisionTreeClassifier().fit(pu_train_data)
        pu_test_data_preRes_DT = pubDT.transform(pu_test_data.select("features")).join(pu_test_data, 'features', "left_outer") 

        predict_to_0_DT = pu_test_data_preRes_DT.rdd.filter(lambda x: (x.pair.split('-')[0] != 'day123' and x.prediction == 0.0))
        predict_to_0_prob_DT = predict_to_0_DT.map(lambda x: x.probability[0]).collect()
        
        prob_DT_max = max(predict_to_0_prob_DT)
        prob_DT_mean = sum(predict_to_0_prob_DT)/len(predict_to_0_prob_DT)
        print(len(predict_to_0_prob_DT), prob_DT_max, min(predict_to_0_prob_DT), prob_DT_mean)
        log_file3.write("len(predict_to_0_prob_DT) ++ max(predict_to_0_prob_DT) ++ min(predict_to_0_prob_DT) ++ mean(predict_to_0_prob_DT) \n")
        log_file3.write(str(len(predict_to_0_prob_DT)) + ', ' + str(prob_DT_max) + ', ' + str(min(predict_to_0_prob_DT)) + ', ' + str(prob_DT_mean) + '\n')
        log_file3.write(time.asctime() + '\n')
        
        #RN_DT_max = pu_test_data_preRes_DT.rdd.filter(lambda x: x.probability[0] >= prob_DT_max).map(lambda x: x.features).collect()
        RN_DT_mean = pu_test_data_preRes_DT.rdd.filter(lambda x: x.probability[0] > prob_DT_mean).map(lambda x: x.features).collect()
        #log_file3.write('len(RN_DT_max): '+str(len(RN_DT_max))+'\n')
        log_file3.write("len(RN_DT_mean):" + str(len(RN_DT_mean))+'\n')
        #print('len(RN_DT_max): '+str(len(RN_DT_max))+'\n')
        print("len(RN_DT_mean):" + str(len(RN_DT_mean))+'\n')
        #sc.parallelize(RN_DT_max).saveAsTextFile(base_path + "RN_blta/RN_DT_max-" + str(blta))
        sc.parallelize(RN_DT_mean).saveAsTextFile(base_path + "RN_blta/RN_DT_mean-" + str(blta))
        """
        ### RandomForestClassifier
        log_file3.write("###### RandomForestClassifier ######\n")
        pubRF = RandomForestClassifier().fit(pu_train_data)
        pu_test_data_preRes_RF = pubRF.transform(pu_test_data.select("features")).join(pu_test_data, 'features', "left_outer")

        predict_to_0_RF = pu_test_data_preRes_RF.rdd.filter(lambda x: (x.pair.split('-')[0] != 'day123' and x.prediction == 0.0))
        predict_to_0_prob_RF = predict_to_0_RF.map(lambda x: x.probability[0]).collect()
        
        prob_RF_max = max(predict_to_0_prob_RF)
        prob_RF_mean = sum(predict_to_0_prob_RF)/len(predict_to_0_prob_RF)
        print(len(predict_to_0_prob_RF), prob_RF_max, min(predict_to_0_prob_RF), prob_RF_mean)
        log_file3.write("len(predict_to_0_prob_RF) ++ max(predict_to_0_prob_RF) ++ min(predict_to_0_prob_RF) ++ mean(predict_to_0_prob_RF) \n")
        log_file3.write(str(len(predict_to_0_prob_RF)) + ', ' + str(prob_RF_max) + ', ' + str(min(predict_to_0_prob_RF)) + ', ' + str(prob_RF_mean) + '\n')
        log_file3.write(time.asctime() + '\n')
        
        #RN_RF_max = pu_test_data_preRes_RF.rdd.filter(lambda x: x.probability[0] >= prob_RF_max).map(lambda x: x.features).collect()
        RN_RF_mean = pu_test_data_preRes_RF.rdd.filter(lambda x: x.probability[0] > prob_RF_mean).map(lambda x: x.features).collect()
        #log_file3.write('len(RN_RF_max): '+str(len(RN_RF_max))+'\n')
        log_file3.write("len(RN_RF_mean):" + str(len(RN_RF_mean))+'\n')
        #print('len(RN_RF_max): '+str(len(RN_RF_max))+'\n')
        print("len(RN_RF_mean):" + str(len(RN_RF_mean))+'\n')
        #sc.parallelize(RN_RF_max).saveAsTextFile(base_path + "RN_blta/RN_RF_max-" + str(blta))
        sc.parallelize(RN_RF_mean).saveAsTextFile(base_path + "RN_blta/RN_RF_mean-" + str(blta))"""

Tue Feb 28 13:42:53 2017

Tue Feb 28 13:43:37 2017
Tue Feb 28 13:45:09 2017
148078 1.0 0.693548387097 0.758206792855
len(RN_DT_mean):899906



In [11]:

for blta in [0.9, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]: # 0.9, 0.7, 0.6, 0.5,
    with open('qq_PU_blta_21.log', 'a+') as log_file4:
        log_file4.write(time.asctime() + '\n')
        log_file4.write("===Predict with RN===\n" + '\n')
        log_file4.write( "==========blta: " + str(blta) + "============\n")
        print(time.asctime())
        """
        # LR
        log_file4.write('###### LogisticRegression #######\n')
        RN_LR = sc.parallelize(sc.textFile(base_path + "RN_blta/RN_LR_mean-" + str(blta)).map(lambda x: eval(x)).takeSample(False, 240000))
        
        PU_train_data_LR = train_positive_pairs.map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
        .union(RN_LR.map(lambda x: Row(label=0.0, features=Vectors.dense(x)))).toDF()  ###### RN feature dense
        
        lr = LogisticRegression().fit(PU_train_data_LR)
        predict_4th_day_LR = lr.transform(test_data_4thday.select('features'))
        
        y_predict_PU_LR = [i.prediction for i in predict_4th_day_LR.select('prediction').collect()]
        
        recall_score_PU_LR = recall_score(y_label, y_predict_PU_LR)
        precision_score_PU_LR = precision_score(y_label, y_predict_PU_LR)
        f1_score_PU_LR = f1_score(y_label, y_predict_PU_LR)
        tp_LR = 0
        tn_LR = 0
        for i in range(test_data_4thday_length):
            if y_label[i] == 1.0 and y_predict_PU_LR[i] == 1.0:
                tp_LR += 1
            if y_label[i] == 0.0 and y_predict_PU_LR[i] == 0.0:
                tn_LR += 1
        accuracy_LR = (tp_LR + tn_LR)/test_data_4thday_length
        
        print(recall_score_PU_LR, precision_score_PU_LR, f1_score_PU_LR, accuracy_LR)

        log_file4.write('\n')
        log_file4.write("recall_score_PU_LR: " + str(recall_score_PU_LR) + '\n')
        log_file4.write("precision_score_PU_LR: " + str(precision_score_PU_LR) + '\n')
        log_file4.write("f1_score_PU_LR: " + str(f1_score_PU_LR) + '\n')
        log_file4.write("accuracy_LR: " + str(accuracy_LR) + '\n')
        log_file4.write(time.asctime() + '\n')
        log_file4.write('\n')

        print(time.asctime())
        
        # NB
        log_file4.write('###### NaiveBayes #######\n')
        RN_NB = sc.parallelize(sc.textFile(base_path + "RN_blta/RN_NB_mean-" + str(blta)).map(lambda x: eval(x)).takeSample(False, 240000))
        
        PU_train_data_NB = train_positive_pairs.map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
        .union(RN_NB.map(lambda x: Row(label=0.0, features=Vectors.dense(x)))).toDF()  ###### RN feature dense
        
        NB = NaiveBayes().fit(PU_train_data_NB)
        predict_4th_day_NB = NB.transform(test_data_4thday.select('features'))
        
        y_predict_PU_NB = [i.prediction for i in predict_4th_day_NB.select('prediction').collect()]
        
        recall_score_PU_NB = recall_score(y_label, y_predict_PU_NB)
        precision_score_PU_NB = precision_score(y_label, y_predict_PU_NB)
        f1_score_PU_NB = f1_score(y_label, y_predict_PU_NB)
        tp_NB = 0
        tn_NB = 0
        for i in range(test_data_4thday_length):
            if y_label[i] == 1.0 and y_predict_PU_NB[i] == 1.0:
                tp_NB += 1
            if y_label[i] == 0.0 and y_predict_PU_NB[i] == 0.0:
                tn_NB += 1
        accuracy_NB = (tp_NB + tn_NB)/test_data_4thday_length
        
        print(recall_score_PU_NB, precision_score_PU_NB, f1_score_PU_NB, accuracy_NB)

        log_file4.write('\n')
        log_file4.write("recall_score_PU_NB: " + str(recall_score_PU_NB) + '\n')
        log_file4.write("precision_score_PU_NB: " + str(precision_score_PU_NB) + '\n')
        log_file4.write("f1_score_PU_NB: " + str(f1_score_PU_NB) + '\n')
        log_file4.write("accuracy_NB: " + str(accuracy_NB) + '\n')
        log_file4.write(time.asctime() + '\n')
        log_file4.write('\n')

        print(time.asctime())"""
        
        # DT
        log_file4.write('###### DecisionTreeClassifier #######\n')
        RN_DT = sc.parallelize(sc.textFile(base_path + "RN_blta/RN_DT_mean-" + str(blta)).map(lambda x: eval(x)).takeSample(False, 240000))
        print(train_positive_pairs.count(), RN_DT.count())
        PU_train_data_DT = train_positive_pairs.map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
        .union(RN_DT.map(lambda x: Row(label=0.0, features=Vectors.dense(x)))).toDF()  ###### RN feature dense
        
        DT = DecisionTreeClassifier().fit(PU_train_data_DT)
        predict_4th_day_DT = DT.transform(test_data_4thday.select('features'))
        
        y_predict_PU_DT = [i.prediction for i in predict_4th_day_DT.select('prediction').collect()]
        
        recall_score_PU_DT = recall_score(y_label, y_predict_PU_DT)
        precision_score_PU_DT = precision_score(y_label, y_predict_PU_DT)
        f1_score_PU_DT = f1_score(y_label, y_predict_PU_DT)
        tp_DT = 0
        tn_DT = 0
        for i in range(test_data_4thday_length):
            if y_label[i] == 1.0 and y_predict_PU_DT[i] == 1.0:
                tp_DT += 1
            if y_label[i] == 0.0 and y_predict_PU_DT[i] == 0.0:
                tn_DT += 1
        accuracy_DT = (tp_DT + tn_DT)/test_data_4thday_length
        
        print(recall_score_PU_DT, precision_score_PU_DT, f1_score_PU_DT, accuracy_DT)

        log_file4.write('\n')
        log_file4.write("recall_score_PU_DT: " + str(recall_score_PU_DT) + '\n')
        log_file4.write("precision_score_PU_DT: " + str(precision_score_PU_DT) + '\n')
        log_file4.write("f1_score_PU_DT: " + str(f1_score_PU_DT) + '\n')
        log_file4.write("accuracy_DT: " + str(accuracy_DT) + '\n')
        log_file4.write(time.asctime() + '\n')
        log_file4.write('\n')

        print(time.asctime())
        """
        # RF
        log_file4.write('###### RandomForestClassifier #######\n')
        RN_RF = sc.parallelize(sc.textFile(base_path + "RN_blta/RN_RF_mean-" + str(blta)).map(lambda x: eval(x)).takeSample(False, 240000))
        
        PU_train_data_RF = train_positive_pairs.map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))\
        .union(RN_RF.map(lambda x: Row(label=0.0, features=Vectors.dense(x)))).toDF()  ###### RN feature dense
        
        RF = RandomForestClassifier().fit(PU_train_data_RF)
        predict_4th_day_RF = RF.transform(test_data_4thday.select('features'))
        
        y_predict_PU_RF = [i.prediction for i in predict_4th_day_RF.select('prediction').collect()]
        
        recall_score_PU_RF = recall_score(y_label, y_predict_PU_RF)
        precision_score_PU_RF = precision_score(y_label, y_predict_PU_RF)
        f1_score_PU_RF = f1_score(y_label, y_predict_PU_RF)
        tp_RF = 0
        tn_RF = 0
        for i in range(test_data_4thday_length):
            if y_label[i] == 1.0 and y_predict_PU_RF[i] == 1.0:
                tp_RF += 1
            if y_label[i] == 0.0 and y_predict_PU_RF[i] == 0.0:
                tn_RF += 1
        accuracy_RF = (tp_RF + tn_RF)/test_data_4thday_length
        
        print(recall_score_PU_RF, precision_score_PU_RF, f1_score_PU_RF, accuracy_RF)

        log_file4.write('\n')
        log_file4.write("recall_score_PU_RF: " + str(recall_score_PU_RF) + '\n')
        log_file4.write("precision_score_PU_RF: " + str(precision_score_PU_RF) + '\n')
        log_file4.write("f1_score_PU_RF: " + str(f1_score_PU_RF) + '\n')
        log_file4.write("accuracy_RF: " + str(accuracy_RF) + '\n')
        log_file4.write(time.asctime() + '\n')
        log_file4.write('\n')

        print(time.asctime())
        """
        

Tue Feb 28 14:20:02 2017
235586 240000
0.995660180547 0.639927164377 0.779108493347 0.7219145845622982
Tue Feb 28 14:23:31 2017
Tue Feb 28 14:23:31 2017
235586 240000
0.995428380101 0.645554080124 0.783192834563 0.728543066100867
Tue Feb 28 14:27:08 2017
Tue Feb 28 14:27:08 2017
235586 240000
0.997836529175 0.617233303594 0.762688925089 0.6941447355901886
Tue Feb 28 14:30:37 2017
Tue Feb 28 14:30:37 2017
235586 240000
0.995801836375 0.508968004792 0.673632950318 0.5247283591178094
Tue Feb 28 14:34:07 2017
Tue Feb 28 14:34:07 2017
235586 240000
0.996651771342 0.523775040606 0.686677343365 0.5520097936607613
Tue Feb 28 14:37:46 2017
Tue Feb 28 14:37:46 2017
235586 240000
0.994179233256 0.517041382867 0.680286915221 0.5397233163974045
Tue Feb 28 14:41:15 2017
Tue Feb 28 14:41:15 2017
235586 240000
0.995582913732 0.522202558664 0.685071710553 0.5491427375311602
Tue Feb 28 14:44:46 2017


In [5]:
2*0.771290227036*0.00733894703822/(0.771290227036+ 0.00733894703822) # 0.0145395480051

0.014539548005105499

In [2]:
a = [('aa',[0,1,2,3,0]), ('aa',[0,1,2,4,0])]
c = [('abs',[0,0,0,0,0]), ('aas',[0,1,1,0,0])]

In [3]:
b = sc.parallelize(a).map(lambda x: Row(label=0.0, features=Vectors.dense(x[1]))).union((sc.parallelize(c).map(lambda x: Row(label=1.0, features=Vectors.dense(x[1])))))

In [11]:
base_path = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_prediction/"

sc.parallelize(b.takeSample(False, 2)).saveAsTextFile(base_path + "xxx")

In [4]:
b.collect()

[Row(features=DenseVector([0.0, 1.0, 2.0, 3.0, 0.0]), label=0.0),
 Row(features=DenseVector([0.0, 1.0, 2.0, 4.0, 0.0]), label=0.0),
 Row(features=DenseVector([0.0, 0.0, 0.0, 0.0, 0.0]), label=1.0),
 Row(features=DenseVector([0.0, 1.0, 1.0, 0.0, 0.0]), label=1.0)]

In [8]:
b.unpersist()

UnionRDD[4] at union at NativeMethodAccessorImpl.java:-2

In [9]:
b.collect()

[Row(features=DenseVector([0.0, 1.0, 2.0, 3.0, 0.0]), label=0.0),
 Row(features=DenseVector([0.0, 1.0, 2.0, 4.0, 0.0]), label=0.0),
 Row(features=DenseVector([0.0, 0.0, 0.0, 0.0, 0.0]), label=1.0),
 Row(features=DenseVector([0.0, 1.0, 1.0, 0.0, 0.0]), label=1.0)]